In [2]:
import pandas as pd
import geopandas as gpd
import json
import numpy as np

from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score

import plotly.express as px

In [3]:
lsoas = gpd.read_file('../geodata/LSOAs.geojson').to_crs(epsg=4326)
lsoas = (
    lsoas[lsoas['LAD11NM'] != 'City of London']
    [['LSOA11CD','geometry']]
    .rename(columns={'LSOA11CD':'lsoa_code'})
)

data = pd.read_csv('../data/predictive_data.csv')

In [ ]:
# Per-LSOA temporal split
def lsoa_wise_temporal_split(df, test_months=3):
    train_list, test_list = [], []
    # for each lsoa and its subdf group
    for lsoa, group in df.groupby('lsoa_code'):
        group = group.sort_values('month')
        # if the lsoa has less than or equal to amount of test months always put in train list
        if group.shape[0] <= test_months:
            train_list.append(group)
        # else there is enough data, so add all but last test_months rows to training
        # add last test_months rows to test
        else:
            train_list.append(group.iloc[:-test_months])
            test_list.append(group.iloc[-test_months:])
    # convert into dfs
    train_df = pd.concat(train_list)
    test_df = pd.concat(test_list)
    return train_df, test_df

In [ ]:
# independent variables
features = [
    'rolling_std_3','rolling_mean_6','rolling_sum_12',
    'Anti-social behaviour', 'Bicycle theft', 'Criminal damage and arson',
    'Drugs', 'Other crime', 'Other theft',
    'Public order', 'Robbery', 'Shoplifting', 'Theft from the person',
    'Vehicle crime', 'Violence and sexual offences',
    'Possession of weapons', 'rolling_mean_3','health_decile_2019',
    'lag_1','lag_2','lag_3','imd_decile_2019','income_decile_2019','employment_decile_2019',
    'crime_decile_2019',
]

# get train and test dfs 
train_df, test_df = lsoa_wise_temporal_split(data, test_months=3)
print(f"Train rows: {len(train_df)}, Test rows: {len(test_df)}")

X_train = train_df[features]
y_train = train_df[['target_1','target_2','target_3']]
X_test  = test_df[features]
y_test  = test_df[['target_1','target_2','target_3']]

In [ ]:
import time
# gradient boost models
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
# random forest model
from sklearn.ensemble import RandomForestRegressor
# adaboost
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# list of regressors to test
regressors = {
    'LightGBM': LGBMRegressor(n_estimators=300, random_state=42, verbose=-1),
    'XGBoost': XGBRegressor(n_estimators=300, random_state=42, verbosity=0),
    'Random_Forest': RandomForestRegressor(n_estimators=100, random_state=42, verbose=False),
    'cat': CatBoostRegressor(iterations=300, learning_rate=0.1, random_state=42, verbose=False),
    'ada': AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=3), n_estimators=100, learning_rate=0.1, random_state=42),
}

# change to choose model
name = 'LightGBM'

# regressors by themselves predicts one month ahead,
# so MultiOutputRegressor allows to predict multiple (all three targets)
rf = MultiOutputRegressor(regressors[name])

t0 = time.perf_counter()    # timer for fitting
rf.fit(X_train, y_train)
fit_time = time.perf_counter() - t0

print(f'Regressor: {name}')
print(f'Fit time: {fit_time:.2f} seconds')

# get predictions for three months into the future
y_pred = rf.predict(X_test)

# print rmse and r^2 for all three months in future
horizons = ['1‑month ahead', '2‑months ahead', '3‑months ahead']
for i, title in enumerate(horizons):
    rmse = np.sqrt(mean_squared_error(y_test.values[:,i], y_pred[:,i]))
    r2   = r2_score(y_test.values[:,i], y_pred[:,i])
    print(f'{title}: RMSE={rmse:.2f}, R²={r2:.3f}')

In [ ]:
# show feature importances
importances = np.mean([est.feature_importances_ for est in rf.estimators_], axis=0)
imp_df = pd.DataFrame({'feature': features, 'importance': importances}).sort_values('importance', ascending=False)
print(imp_df)

In [ ]:
# CREATE VISUALIZATION
vis_df = test_df.copy()
vis_df[['pred_1', 'pred_2', 'pred_3']] = y_pred

# get the predictions from the latest months so that future months are shown
month = 3
latest_preds = (
    vis_df.sort_values('month')
           .groupby('lsoa_code')
           .tail(1)[['lsoa_code', f'pred_{month}']]
           .rename(columns={f'pred_{month}': 'predicted_burglaries'})
)

latest_preds.reset_index(drop=True, inplace=True)

# Merge predictions with GeoData
geo = lsoas.merge(latest_preds, on='lsoa_code', how='inner')

# Convert to JSON
geojson = json.loads(geo.to_json())

# Plot choropleth (works in Jupyter)
fig = px.choropleth_map(
    geo,
    geojson=geojson,
    locations='lsoa_code',
    featureidkey="properties.lsoa_code",
    color='predicted_burglaries',
    range_color=(0, geo['predicted_burglaries'].max()),
    color_continuous_scale="OrRd",
    map_style="open-street-map",
    zoom=9,
    center={"lat": 51.5072, "lon": -0.1276},
    opacity=0.6,
    height=600
)

fig.update_layout(title=f'Predicted Burglary Count using {name} ({month}-month horizon) by LSOA')
fig.show()